In [5]:
# import libraries

import numpy as np
import pandas as pd
from tsfresh.feature_extraction import extract_features, EfficientFCParameters
from datetime import datetime
import glob
import shutil
import glob

In [8]:
file_list = glob.glob('Base_Prototipo/*')
file_list.sort()

In [9]:
# Gambiarra pra rodar só os dados de um dia
matheus = []
thiago = []
for f in file_list:
    dia = f.split('/')[1].split('-')[0]
    
    if dia == '18':
        matheus.append(f)
    else:
        thiago.append(f)

file_list = thiago # modificar essa linha para rodar o dia 19

In [12]:
df = pd.DataFrame()

for f in file_list:
    
    new_name = "Arquivos_Processados/" + f.split('/')[-1]

    data = np.genfromtxt(f, delimiter=',')
    
    df2 = pd.DataFrame(data, columns=['id', 'time', 
                                     'Sensor_0', 'Sensor_1',
                                     'Sensor_2', 'Sensor_3',
                                     'Sensor_4', 'Sensor_5'])
    
    inicio = datetime.now()

    extracted_features = extract_features(df2, column_id="id", column_sort="time",
                                default_fc_parameters = EfficientFCParameters(),
                                chunksize=100,
                                n_jobs=4,
                                disable_progressbar=False)
    
    df = pd.concat([df, extracted_features])
    
    print('--->', f)
    print(datetime.now() - inicio)
    print(' ')
    df.to_csv('extracted_features.csv')

    shutil.move(f, new_name)
    
    
df = df.dropna(axis='columns')
df.to_csv('extracted_features.csv')

Feature Extraction:   0%|          | 0/40 [00:00<?, ?it/s]

'Arquivos_Processados/19-Jan-2022-08.47.14.csv'